In [1]:
import math
import numpy as np
import pandas as pd
import sys
import re
from functions import *
from mining_functions import *
from collections import Counter
from matplotlib.backends.backend_pdf import PdfPages
np.set_printoptions(precision=2)
pd.set_option('precision', 2)
# %matplotlib inline
matplotlib.style.use('ggplot')

In [2]:
PATH = '/Google Drive/Jonathan Sarah Ido folder/data/CCK/'
def get_path(path = PATH):
    if os.name == 'posix':
        return os.environ['HOME']+path #'/Google Drive/Jonathan Sarah Ido folder/data/CCK/'
    elif os.name == 'nt':
        return os.environ['USERPROFILE']+ path.replace('/','\\') #'\\Google Drive\Jonathan Sarah Ido folder\data\CCK\\'
    else:
        raise Exception('OS not recongnized. I\'m confused.')
gitpath = '/Documents/git/Phet-log-analyzer/cck/raw_data_parsing_check/'
df = pd.read_csv(get_path(path = gitpath) + 'phet_cck_user_actions+sophistication_WITHPAUSE_more_circuit_info.txt',index_col=False)
# dfx = pd.read_csv('C:\Users\Sarah\Documents\git\Phet-log-analyzer\cck\\raw_data_parsing_check\phet_cck_user_actions+sophistication_WITHPAUSE_more_circuit_info.txt',index_col=False)
df["student"] = df["student"].astype('category')
df["Family"]=df["Family"].str.capitalize()
df["Family_tool"]=df["Family_tool"].str.capitalize()
df["Family_default"]=df["Family_default"].str.capitalize()
df["Family_both"]=df["Family_both"].str.capitalize()

df_scores = pd.read_csv(data_path + 'MATCHING_phet_cck_user_data_anonymized.txt')
df_scores["student"] = df_scores["student"].astype('category')

### Functions used to calculate information gain, plot use, etc...

In [3]:
def add_text(attribute,family_category,N, shortest_seq_length, longest_seq_length,B):
    text = """Showing sequences for students split by {0}, using the categories {1}.
            Removed sequences used by less than {2}.
            Found sequences of lenght {3} to {4}.
            Using {5} time bins""".format(attribute,family_category,N, shortest_seq_length, longest_seq_length,B)
    fig = plt.figure(figsize=(10, 4))
    ax = plt.gca()
    ax.text(0.5,0.5,text,
        horizontalalignment='center',
        verticalalignment='center',
        fontsize = 15)
    plt.axis('off')
    return fig

In [4]:
def rank_sequences(sequence_counts,B,axesnum=None):
    ranks = []
    for seq,counts in sequence_counts.iteritems():
#         if np.sum(counts)>0:
        ranks.append((seq,calc_infogain(counts,B,axesnum)))
    return sorted(ranks, key=lambda tup: tup[1])

def get_top_seqs(ranks,N):
    return ranks[-N:]

### First we get the raw count of unique students using a sequence per time bin

In [5]:
print list(df_scores.columns.values)

['Wave', 'scaffolding', 'how many PhETs in the past?', 'how comfortable with phets?', 'p100', 'used this circuit sim before?', 'COMPLETEDNESS', 'PRIORKNOWLEDGE', 'COMPLEXITY', 'DIAGRAMS', 'NUMBERSANDCALCULATIONSONLYINATLEASTONEANSWER', 'pre', 'post t1', 'post t2', 'post identical to pre', 'z post identical to pre', 'z pre', 'z post t1', 'z post t2', 'T0_PoCC', 'T1_PoCC', 'T2_PoCC', 'Avg_Mast_Or', 'Perceived Value ', 'Clustergroups', 'fourgroups', 'student', 'Anon Student Id']


In [6]:
def label_learning (median,row,column):
    if row[column] >= median: return 'high'
    else: return 'low'

median_pre = np.median(df_scores['pre'])
df_scores['incoming_knowledge'] = df_scores.apply (lambda row: label_learning (median_pre,row,"pre"),axis=1)

median_T0_PoCC = np.median(df_scores['T0_PoCC'])
df_scores['incoming_attitude'] = df_scores.apply (lambda row: label_learning (median_T0_PoCC,row,"T0_PoCC"),axis=1)

In [11]:
students = get_students()
CUT_OFF_SEQ_USE = 0.2 #we keep only sequences used once by at least 10% of students
N = int(CUT_OFF_SEQ_USE*len(students))
shortest_seq_length = 3
longest_seq_length = 12
B = 4  #number of bins

### PARAMETERS
attributes = [('learning2','high', 'low'),('scaffolding','scaff', 'not'),('incoming_knowledge','high', 'low'),('incoming_attitude','high', 'low')]
# attributes = [('learning2','high', 'low')]

family_categories = ["Family","Family_tool","Family_default","Family_both"]
# family_categories = ["Family"]

# sequence_counts = get_sequence_use_by_timebin(df,students,family_category,B,attribute,level1,level2,shortest_seq_length,longest_seq_length,N)
# for k,v in sequence_counts.iteritems():
#     print k,v, np.sum(v)

### We can calculate the information gain of each sequence by time bin

In [12]:
parse_axis = {0:'time',1:'group',None:'time and group'}

pdf = PdfPages('infogain_results.pdf')

for attribute,level1,level2 in attributes:
    for family_category in family_categories:
        print "For attribute {0}, categories {1}".format(attribute,family_category)
        pdf.savefig(add_text(attribute,family_category,N, shortest_seq_length, longest_seq_length,B))
        sequence_counts = get_sequence_use_by_timebin(df,students,family_category,B,attribute,level1,level2,shortest_seq_length,longest_seq_length,N)
        ylabels = [level1,level2]
        for axis in [0,1,None]:
            tops = get_top_seqs(rank_sequences(sequence_counts,B,axis),1)
            for seq,infogain in tops:
                title = '{0}: infogain {1} by {2}'.format(seq,round(infogain,3),parse_axis[axis])
                plot = plot_heat_map(sequence_counts[seq],title, ylabels)
                pdf.savefig( plot )
pdf.close()

For attribute learning2, categories Family
Getting sequence use over 4 time bins for 96 students split by learning2. 
            Keeping only sequences used once by at least 19 students.
For attribute learning2, categories Family_tool
Getting sequence use over 4 time bins for 96 students split by learning2. 
            Keeping only sequences used once by at least 19 students.
For attribute learning2, categories Family_default
Getting sequence use over 4 time bins for 96 students split by learning2. 
            Keeping only sequences used once by at least 19 students.
For attribute learning2, categories Family_both
Getting sequence use over 4 time bins for 96 students split by learning2. 
            Keeping only sequences used once by at least 19 students.


C:\Users\Sarah\Anaconda2\lib\site-packages\matplotlib\pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


For attribute scaffolding, categories Family
Getting sequence use over 4 time bins for 96 students split by scaffolding. 
            Keeping only sequences used once by at least 19 students.
For attribute scaffolding, categories Family_tool
Getting sequence use over 4 time bins for 96 students split by scaffolding. 
            Keeping only sequences used once by at least 19 students.
For attribute scaffolding, categories Family_default
Getting sequence use over 4 time bins for 96 students split by scaffolding. 
            Keeping only sequences used once by at least 19 students.
For attribute scaffolding, categories Family_both
Getting sequence use over 4 time bins for 96 students split by scaffolding. 
            Keeping only sequences used once by at least 19 students.
For attribute incoming_knowledge, categories Family
Getting sequence use over 4 time bins for 96 students split by incoming_knowledge. 
            Keeping only sequences used once by at least 19 students.
For attr